In [32]:
import nibabel as nib
import numpy as np
import pandas as pd

import os
import re
np.printoptions(precision=3)
pd.set_option('display.max_rows', 500)

In [33]:
def volumen(basepath_label, model):
    results = []

    for filename in os.listdir(basepath_label):
        match = re.search(r'([A-Z]+)_(\d+)([A-Z])\.nii\.gz', filename)
        if not match:
            continue

        initial, MR, day = match.groups()
        path_label = os.path.join(basepath_label, filename)
        img = nib.load(path_label)
        label_data = img.get_fdata()
        vox = img.header.get_zooms()
        vox_x, vox_y = vox[:2]
        vox_z = 5  # 5mm slice thickness
        

        # Calculate area for all frames and collect in a list
        areas = []
        for frame in range(label_data.shape[2]):
            mask_gallbladder = label_data[:, :, frame] == 1  # Get mask where segmentation label is 1
            area_gallbladder = np.sum(mask_gallbladder) * vox_x * vox_y * vox_z  # Calculate area in mm^3
            area_gallbladder_mL = area_gallbladder / 1000
            areas.append(area_gallbladder_mL)
        print(f'Areas: {areas}')
        # Calculate mean area for this MR
        MR_area = np.sum(areas)

        results.append({
            'Model': model,
            'Subject': initial,
            'Day': day,
            'MR': MR,
            'Area': MR_area
        })

    results_df = pd.DataFrame(results)
    print(results_df.sort_values(by=['Subject', 'Day', 'MR']))
    print(f'Voxel size: {vox_x} x {vox_y} x {vox_z} mm^3')
    return results_df.sort_values(by=['Subject', 'Day', 'MR'])

In [34]:
def difference(results_true, results_pred):
    results_df = pd.merge(results_true, results_pred, on=['Subject', 'Day', 'MR'], suffixes=('_true', '_pred'))
    results_df['Difference'] = results_df['Area_true'] - results_df['Area_pred']
    
    return results_df[['Subject', 'Day', 'MR', 'Difference']]

In [35]:
def save_to_excel(true, pred1, pred2, diff1, diff2, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        true.to_excel(writer, sheet_name='Ground True Labels', index=False)
        pred1.to_excel(writer, sheet_name='Scanner', index=False)
        pred2.to_excel(writer, sheet_name='Calculated', index=False)
        diff1.to_excel(writer, sheet_name='Difference Scanner', index=False)
        diff2.to_excel(writer, sheet_name='Difference Calculated', index=False)
    print(f'Data saved to {filename}')



In [36]:
basepath_true = '/home/klinfys/Desktop/nnUNet_raw/Dataset333_MonaiLabel/labelsTs'
basepath_pred1 = '/home/klinfys/Desktop/nnUNet_raw/Dataset333_MonaiLabel/imagesTs_pred_3d_fullres'
basepath_pred2 = '/home/klinfys/Desktop/nnUNet_raw/Dataset222_T2star/imagesTs_pred_3d_fullres'

model_true = 'Ground truth label'
model_pred1 = 'nnUNet prediction: Scanner'
model_pred2 = 'nnUNet prediction: Calculated'


results_true = volumen(basepath_true, model_true)
results_pred1 = volumen(basepath_pred1, model_pred1)
results_pred2 = volumen(basepath_pred2, model_pred2)
diff_df1 = difference(results_true, results_pred1)
diff_df2 = difference(results_true, results_pred2)
filename = 'Gallbladder_volume.xlsx'
save_to_excel(results_true,results_pred1, results_pred2, diff_df1, diff_df2, filename)


Areas: [2.060150146484375, 2.35760498046875, 2.941497802734375, 2.765228271484375, 0.363555908203125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.4847412109375, 0.958465576171875, 1.1898193359375, 1.1016845703125, 1.861846923828125, 2.48980712890625, 3.28302001953125, 1.4542236328125, 1.23388671875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.544891357421875, 2.985565185546875, 1.82879638671875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.0, 0.0, 0.68304443359375, 2.897430419921875, 3.822845458984375, 4.131317138671875, 3.844879150390625, 2.126251220703125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.751678466796875, 3

Areas: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5423583984375, 2.93048095703125, 4.03216552734375, 3.8338623046875, 1.41015625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [1.178802490234375, 0.980499267578125, 0.892364501953125, 0.969482421875, 1.60845947265625, 2.192352294921875, 1.112701416015625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.187286376953125, 1.421173095703125, 1.575408935546875, 1.883880615234375, 1.905914306640625, 2.53387451171875, 1.972015380859375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.561859130859375, 1.43218994140625, 2.831329345703125, 4.05419921875, 1.024566650390625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Areas: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.